In [4]:
import numpy as np

# .npy 파일 경로
file_path = './intersection/tracked_data.npy'  # 파일 경로를 적절히 설정하세요.

# .npy 파일 불러오기
data = np.load(file_path, allow_pickle=True)

fps=15

In [5]:
# 차량 등장 횟수 계산
vehicle_count = {}

for entry in data:
    for vehicle in entry.keys():
        if vehicle in vehicle_count:
            vehicle_count[vehicle] += 1
        else:
            vehicle_count[vehicle] = 1


filtered_data = []
for entry in data:
    filtered_entry = {vehicle: entry[vehicle] for vehicle in entry.keys() if vehicle_count[vehicle] > 2*fps}
    if filtered_entry:  # 필터링된 결과가 비어있지 않으면 추가
        filtered_data.append(filtered_entry)

# 결과 출력
print("필터링된 데이터:")
for entry in filtered_data:
    print(entry)

필터링된 데이터:
{'vehicle0': array([877, 599, 961, 666])}
{'vehicle0': array([876.8434 , 598.66003, 961.2958 , 666.40247], dtype=float32)}
{'vehicle0': array([876.28925, 599.62616, 960.51245, 667.974  ], dtype=float32)}
{'vehicle0': array([876.78   , 599.68677, 960.82074, 668.49316], dtype=float32)}
{'vehicle0': array([876.16736, 603.43915, 960.81573, 671.91797], dtype=float32)}
{'vehicle0': array([875.56836, 607.20233, 960.2066 , 675.32   ], dtype=float32)}
{'vehicle0': array([875.4232, 607.8048, 959.9661, 675.8894], dtype=float32)}
{'vehicle0': array([874.73083, 607.85156, 959.7137 , 676.26416], dtype=float32)}
{'vehicle0': array([873.74854, 606.61304, 958.78455, 674.97174], dtype=float32)}
{'vehicle0': array([873.7204 , 605.97406, 958.69214, 674.11646], dtype=float32)}
{'vehicle0': array([872.7603 , 604.4907 , 958.9463 , 672.78784], dtype=float32)}
{'vehicle0': array([871.90027, 604.2097 , 958.3667 , 673.0696 ], dtype=float32)}
{'vehicle0': array([871.29895, 604.1583 , 958.49384, 673.2984

In [6]:
vehicle_count = {}

for entry in filtered_data:
    for vehicle in entry.keys():
        if vehicle in vehicle_count:
            vehicle_count[vehicle] += 1
        else:
            vehicle_count[vehicle] = 1

# 결과 출력
for vehicle, count in vehicle_count.items():
    print(f"{vehicle}가 {count}번 등장했습니다.")

vehicle0가 150번 등장했습니다.
vehicle4가 35번 등장했습니다.
vehicle6가 89번 등장했습니다.
vehicle15가 48번 등장했습니다.


In [7]:
np.save('tracked_data.npy', filtered_data)

In [8]:
def calculate_area(bbox):
    x_min, y_min, x_max, y_max = bbox
    return (x_max - x_min) * (y_max - y_min)

def intersect(bbox1, bbox2):
    x1_min, y1_min, x1_max, y1_max = bbox1
    x2_min, y2_min, x2_max, y2_max = bbox2

    # 겹치는 부분의 좌표 계산
    intersect_x_min = max(x1_min, x2_min)
    intersect_y_min = max(y1_min, y2_min)
    intersect_x_max = min(x1_max, x2_max)
    intersect_y_max = min(y1_max, y2_max)

    if intersect_x_max > intersect_x_min and intersect_y_max > intersect_y_min:
        return (intersect_x_min, intersect_y_min, intersect_x_max, intersect_y_max)
    return None  # 겹치지 않음

def subtract_intersection(bbox1, intersection):
    if intersection is None:
        return [bbox1]  # 겹치지 않으면 원래 bbox를 리스트로 반환

    x1_min, y1_min, x1_max, y1_max = bbox1
    ix_min, iy_min, ix_max, iy_max = intersection

    new_boxes = []

    # 겹치지 않는 부분 추가
    if y1_min < iy_min:
        new_boxes.append([x1_min, y1_min, x1_max, iy_min])  # 아래쪽 부분

    if y1_max > iy_max:
        new_boxes.append([x1_min, iy_max, x1_max, y1_max])  # 위쪽 부분

    if x1_min < ix_min:
        new_boxes.append([x1_min, max(y1_min, iy_min), ix_min, min(y1_max, iy_max)])  # 왼쪽 부분

    if x1_max > ix_max:
        new_boxes.append([ix_max, max(y1_min, iy_min), x1_max, min(y1_max, iy_max)])  # 오른쪽 부분

    return new_boxes

def select_largest_non_overlapping(bounding_boxes):
    remaining_boxes = bounding_boxes.copy()
    selected_boxes = []

    while remaining_boxes:
        # 가장 큰 면적의 bounding box 선택
        largest_box_key = max(remaining_boxes, key=lambda k: calculate_area(remaining_boxes[k]))
        largest_box = remaining_boxes.pop(largest_box_key)

        # 선택된 박스를 결과에 추가
        selected_boxes.append((largest_box_key, largest_box))

        # 나머지 bounding box에서 겹치는 부분 제거
        new_remaining_boxes = {}
        for key in list(remaining_boxes.keys()):
            intersection = intersect(largest_box, remaining_boxes[key])
            if intersection:
                # 겹치는 부분이 있을 경우 남은 부분을 추가
                new_boxes = subtract_intersection(remaining_boxes[key], intersection)
                for idx, new_box in enumerate(new_boxes):
                    # 기존 key에 idx를 추가해 고유하게 만듦
                    new_remaining_boxes[f"{key}_{idx}"] = new_box
            else:
                # 겹치지 않으면 그대로 추가
                new_remaining_boxes[key] = remaining_boxes[key]

        # 새로 남은 bounding box를 업데이트
        remaining_boxes = new_remaining_boxes

    # 최종 결과 생성: 각 vehicle에 대해 하나의 bounding box만 남기기
    final_boxes = {}
    for key, box in selected_boxes:
        vehicle_id = key.split('_')[0]  # 원래 vehicle ID를 복원
        if vehicle_id not in final_boxes:
            final_boxes[vehicle_id] = box
        else:
            # 이미 존재하는 vehicle이라면 겹치는 부분을 조정
            existing_box = final_boxes[vehicle_id]
            intersection = intersect(existing_box, box)
            if intersection:
                # 겹치는 부분이 있다면 겹치지 않는 부분 추가
                new_boxes = subtract_intersection(box, intersection)
                for nb in new_boxes:
                    final_boxes[vehicle_id] = nb

    return list(final_boxes.items())


# 예시 bounding_boxes
result=[]
for i in range(len(filtered_data)):
    temp=select_largest_non_overlapping(filtered_data[i])
    temp2={}
    for t in temp:
        temp2[t[0]]=np.array(t[1])
    result.append(temp2)

print(result[20])

{'vehicle0': array([868.7745 , 604.36066, 958.214  , 675.8613 ], dtype=float32)}


In [9]:
np.save('intersection/crop_tracked_data.npy', result)

In [10]:
print(filtered_data[66]) ##기존
print(result[66])  ## 변경본

{'vehicle0': array([835.5767, 600.8801, 958.4849, 698.8851], dtype=float32), 'vehicle4': array([796.8402 , 615.9853 , 835.472  , 644.58575], dtype=float32), 'vehicle6': array([1235.9548,  602.6543, 1475.8478,  693.5863], dtype=float32)}
{'vehicle6': array([1235.9548,  602.6543, 1475.8478,  693.5863], dtype=float32), 'vehicle0': array([835.5767, 600.8801, 958.4849, 698.8851], dtype=float32), 'vehicle4': array([796.8402 , 615.9853 , 835.472  , 644.58575], dtype=float32)}


In [19]:
len(result)

150

In [48]:
# bounding box 딕셔너리 예시

# 최소 넓이 기준 설정
min_area = 2000  # 예시값, 원하는 값으로 설정

new_result=[]
for bounding_boxes in result:
    temp= {key: bbox for key, bbox in bounding_boxes.items() if (bbox[2] - bbox[0]) * (bbox[3] - bbox[1]) > min_area}
    new_result.append(temp)
print(new_result[76])

{'vehicle6': array([1076.9995 ,  604.6071 , 1358.9065 ,  706.14386], dtype=float32), 'vehicle0': array([836.1674 , 600.49097, 961.6817 , 696.5529 ], dtype=float32), 'vehicle4': array([759.0426 , 611.93555, 811.90076, 650.4739 ], dtype=float32)}
